In [2]:
import cv2
import numpy as np
import math
import colorsys

In [23]:
#ランダムで座標と値を発生
#本番はリストで受け取るので、この関数は不要
def getRandom2DPoints(width, height, n):
    np.random.seed(202001)
    margin = 10
    entry_pts = np.random.randint(margin, max(height, width), (n*2, 2)) 
    #第一引数に最小値、第二引数に最大値、第三引数が返り値の配列の形状(100*2(=n*2かけ2)行列)
    
    pts = []
    for i, ep in enumerate(entry_pts):
        #x座標が範囲外でないか：今はseedを大きく設定しているから大丈夫...
        if ep[0] < margin or (width-margin) < ep[0]:
            #ループの先頭に戻ってやり直す
            continue 
            
        #y座標が範囲外でないか
        if ep[1] < margin or (height-margin) < ep[1]:
            continue

        # サークル色をHSVで計算
        # 白っぽいを避けるためHSVで計算
        h = i * math.pi * 2.0 / n
        rgb = colorsys.hsv_to_rgb(h, 1.0, 1.0)
        #値aを0-100の間にする
        a = int(rgb[0] * 100)
        pts.append([ep[0],ep[1],a])
    return    pts

In [13]:
#testcase  seedが20だったら、座標がn*2個できない
#20の値を持った座標を生成
pts = getRandom2DPoints(50, 50, 10)
print(pts)
print(len(pts))

(20, 2)
[[38, 36, 46], [19, 30, 100], [21, 32, 0], [36, 36, 0], [29, 26, 100], [36, 23, 99], [21, 35, 77], [13, 20, 0], [21, 23, 100], [29, 40, 8], [16, 21, 100]]
11


In [19]:
#受け取った3値から平均を求める。平均値を参照して色分けする。
def get_color(a1,a2,a3):
    #平均値を求める
    ave = (a1+a2+a3)/3
    
    #値のとりうる範囲を10分割して、白~黒に色分け
    min = 0
    max = 100
    std = (max-min)/10
    
    r=0
    g=0
    b=0
    if(ave<std): #黒
        r=g=b=0
    elif (ave<std*2): 
        r=g=b=25
    elif (ave<std*3): 
        r=g=b=50
    elif (ave<std*4): 
        r=g=b=75
    elif (ave<std*5): 
        r=g=b=100
    elif (ave<std*6): 
        r=g=b=125
    elif (ave<std*7): 
        r=g=b=150
    elif (ave<std*8): 
        r=g=b=175
    elif (ave<std*9): 
        r=g=b=200
    else: #白
        r=g=b=225
        
    return(r,g,b)

In [20]:
def DelaunayDiagram(imgd, subdiv, array):
    height, width = imgd.shape[:2]

    # ドロネーの三角形
    triangles = subdiv.getTriangleList()
    pols = triangles.reshape(-1, 3, 2)
    inner_pts = np.empty((0, 3, 2))
    
    for pa in pols:
        # フレーム外の点は除外
        cnt = 0
        for p in pa:
            if p[0] < 0 or width <= p[0] :
                break
            if p[1] < 0 or height <= p[1] :
                break
            cnt += 1
        # フレーム内の点は有効
        if cnt == 3:
            inner_pts = np.append(inner_pts, [pa], axis=0)
    
    #rgb値を格納する配列を作成
    color_list = np.zeros((len(inner_pts),3))
    
    #3値を渡して色を設定
    for i in range(len(inner_pts)):
        color_list[i]=get_color(array[(int)(inner_pts[i][0][0])][(int)(inner_pts[i][0][1])],array[(int)(inner_pts[i][1][0])][(int)(inner_pts[i][1][1])],array[(int)(inner_pts[i][2][0])][(int)(inner_pts[i][2][1])])
    
    #図形の描画
    for i in range(len(inner_pts)):
        #三角形の描画
        cv2.fillPoly(imgd, [inner_pts[i].astype(int)], color_list[i], 1)
        #三角形の辺の描画
        cv2.line(imgd,(inner_pts[i][0][0].astype(int),inner_pts[i][0][1].astype(int)),(inner_pts[i][1][0].astype(int),inner_pts[i][1][1].astype(int)),(0,0,0))
        cv2.line(imgd,(inner_pts[i][1][0].astype(int),inner_pts[i][1][1].astype(int)),(inner_pts[i][2][0].astype(int),inner_pts[i][2][1].astype(int)),(0,0,0))
        cv2.line(imgd,(inner_pts[i][2][0].astype(int),inner_pts[i][2][1].astype(int)),(inner_pts[i][0][0].astype(int),inner_pts[i][0][1].astype(int)),(0,0,0))

In [26]:
def main():
    height = 240
    width = 320
    
    #点を第３引数*2個生成
    pts = getRandom2DPoints(width,height,50)
    
    #白紙を作る 第一引数：生成したい配列の形状shape　第二引数：任意の埋める値　第三引数：データの型
    img = np.full((height, width, 3), 255, np.uint8)
    
    rect = (0, 0, width, height)
    # OpenCVのSubdiv2Dクラスを使用
    # 引数：矩形領域を示すタプル(左上のx座標, 左上のy座標, 右下のx座標, 右下のy座標)
    subdiv = cv2.Subdiv2D(rect)

    #(x1,y1,a1)の時、array[x1][y1]=a1とする
    array = np.zeros((width,height))
    
    for p in pts:
        # 対象の点の座標を追加 p[0]:x座標 p[1]:y座標
        subdiv.insert((int(p[0]), int(p[1])))
        array[p[0]][p[1]] = p[2]
        
    DelaunayDiagram(img, subdiv, array)

    for p in pts:
        color = p[2]
        cv2.circle(img, (p[0],p[1]), 4, color, thickness=-1, lineType=cv2.LINE_AA)
   
    cv2.imwrite('test2.png', img)

In [27]:
if __name__ == '__main__':
    main()

<Subdiv2D 0x7ffa4bd7d630>


https://docs.opencv.org/4.x/dc/d0d/tutorial_py_features_harris.html

特徴量抽出

In [10]:
filename = 'data/naubert.jpg'
img = cv2.imread(filename)

#gray画像にconvert
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
gray = np.float32(gray)

# Harisのtheory
dst = cv2.cornerHarris(gray,2,3,0.04)

#result is dilated for marking the corners, not important
dst = cv2.dilate(dst,None)

# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]

cv2.imshow('dst',img)
cv2.waitKey(0) #何かのキーが押されるまで表示
cv2.destroyAllWindows()